In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import hydra
from omegaconf import DictConfig, OmegaConf,open_dict
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from dataset.semKITTI_dataset import SemKITTI_DVPS_Dataset
from diffusers import AutoencoderKL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ldmseg.models import GeneralVAESeg
from ldmseg.trainers import TrainerAE
from ldmseg.utils import prepare_config, Logger, is_main_process
import yaml
import torch.distributed as dist

In [2]:
print("Starting distributed training", "yellow")
dist.init_process_group(backend="nccl",
                                init_method="tcp://127.0.0.1:54286",
                                world_size=1,
                                rank=0)

print("Initialized distributed training", "yellow")

Starting distributed training yellow
Initialized distributed training yellow


In [3]:
vae_encoder = None
# -----------------------
# 实例化模型
# 对于分类任务，将 out_channels 设置为类别数 19
vae = GeneralVAESeg(
    in_channels= 11,  # consider bit encoding
   int_channels= 256,
   out_channels= 11,
   block_out_channels= [32, 64, 128, 256],
   latent_channels= 4,
   num_latents= 2,
   num_upscalers= 2,
   upscale_channels= 256,
   norm_num_groups= 32,
   scaling_factor= 0.2,
   parametrization= 'gaussian',
   act_fn= 'none',
   clamp_output= False,
   freeze_codebook= False,
   num_mid_blocks= 0,
   fuse_rgb= False,
   resize_input= False,
   skip_encoder= False,
    encoder=vae_encoder
)
print(vae)

Interpolation factor:  2
Parametrization:  gaussian
Activation function:  none
GeneralVAESeg(
  (encoder): Sequential(
    (0): Conv2d(11, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): SiLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): SiLU()
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): SiLU()
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): SiLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): Identity()
    (13): GroupNorm(32, 256, eps=1e-06, affine=True)
    (14): SiLU()
    (15): Conv2d(256, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (decoder): Sequential(
    (0): Con

In [4]:
config = "/root/autodl-tmp/latent-diffusion-segmentation/tools/configs/base/base.yaml"
with open(config, 'r', encoding='utf-8') as fin:
    p = yaml.load(fin, Loader=yaml.FullLoader)
    fin.close()
p['name'] = 'segmentation_diffusion'
p['train_kwargs']['train_num_steps'] = 100
p['lr_scheduler_kwargs']["warmup_iters"]=50

In [5]:
config_dis = "/root/autodl-tmp/latent-diffusion-segmentation/tools/configs/distributed/local.yaml"
with open(config_dis, 'r', encoding='utf-8') as inf:
    cfg_dist = yaml.load(inf, Loader=yaml.FullLoader)
    inf.close()
cfd={"distributed":cfg_dist,"gpu":0}

In [6]:
vae = vae.to('cuda')

In [7]:
 trainer = TrainerAE(
         p,
        vae,
     args=cfd, 
     results_folder=p['output_dir'],                        # output directory
        save_and_sample_every=p['eval_kwargs']['vis_every'],        # save and sample every n iters
        cudnn_on=p['train_kwargs']['cudnn'],                        # turn on cudnn
        fp16=p['train_kwargs']['fp16'],                             # turn on floating point 16
    )

Warning -- Using gradient clipping of 3.0
Most likely not enough memory for training
Please use zero redundancy optimizer with adamw
AdamW (
Parameter Group 0
    amsgrad: False
    betas: [0.9, 0.999]
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.0
)
Train transforms Compose(
    RandomHorizontalFlip(p=0.5)
    CropResize(size=(512, 512), crop_mode=None)
    ToTensor
    Identity()
)
Val transforms Compose(
    CropResize(size=(512, 512), crop_mode=None)
    ToTensor
    Identity()
)
Found 801 samples in split train
<bound method SemKITTI_DVPS_Dataset.__getitem__ of <dataset.semKITTI_dataset.SemKITTI_DVPS_Dataset object at 0x7f9ceb6b61d0>>
The dataset contains 801 samples
training for 3 epochs or 100 iters per epoch or 300 iterations
Using lr scheduler warmup with effective lr: 1.000e-04, final lr: 1.000e-06, warmup iters 50


In [8]:
trainer.resume()

No saved model at /root/autodl-tmp/model.pt to resume ...


In [9]:
if p['load_path'] is not None:
    trainer.load(model_path=p['load_path'])  # looks for model at load_path

if p['eval_only']:
    trainer.compute_metrics(['miou', 'pq'])
    

    # train
trainer.train_loop()

# terminate wandb
if p['wandb'] and is_main_process():
    wandb.finish()

KITTI Eval: 100%|██████████| 101/101 [02:33<00:00,  1.52s/it]


KITTI Evaluation Results:
Mean PQ: 0.00%, Mean SQ: 0.00%, Mean RQ: 0.00%
-------------------------
Starting epoch 0
Learning rate is set to: 0.000e+00
Epoch: [0][ 99/100]	Loss 1.1243e+00 (nan)	CE 5.0618e-02 (nan)	KL 2.0941e+05 (1.1969e+05)	Mask 1.0737e+00 (1.5516e+00)
Model saved for run segmentation_diffusion


KITTI Eval: 100%|██████████| 101/101 [02:30<00:00,  1.49s/it]


KITTI Evaluation Results:
Mean PQ: 0.00%, Mean SQ: 0.00%, Mean RQ: 0.00%
Average loss: nan
Epoch took 0:05:05.843230
ETA: 0:15:26.928123
-------------------------
Starting epoch 1
Learning rate is set to: 1.000e-04
Epoch: [1][ 99/100]	Loss 1.0793e+00 (nan)	CE 7.2922e-02 (nan)	KL 2.2446e+05 (2.1894e+05)	Mask 1.0063e+00 (1.0269e+00)
Model saved for run segmentation_diffusion


KITTI Eval: 100%|██████████| 101/101 [02:31<00:00,  1.50s/it]


KITTI Evaluation Results:
Mean PQ: 0.00%, Mean SQ: 0.00%, Mean RQ: 0.00%
Average loss: nan
Epoch took 0:05:04.737811
ETA: 0:06:24.101020
-------------------------
Starting epoch 2
Learning rate is set to: 1.000e-04
Epoch: [2][ 99/100]	Loss 1.0066e+00 (nan)	CE 8.6001e-03 (nan)	KL 2.2321e+05 (2.2420e+05)	Mask 9.9804e-01 (9.8225e-01)
Model saved for run segmentation_diffusion


KITTI Eval: 100%|██████████| 101/101 [02:30<00:00,  1.50s/it]


KITTI Evaluation Results:
Mean PQ: 0.00%, Mean SQ: 0.00%, Mean RQ: 0.00%
Average loss: nan
Epoch took 0:05:05.362331
ETA: 0:00:00


KITTI Eval: 100%|██████████| 101/101 [02:31<00:00,  1.50s/it]


KITTI Evaluation Results:
Mean PQ: 0.00%, Mean SQ: 0.00%, Mean RQ: 0.00%
Finished run segmentation_diffusion and took 0:20:27.551457
